In [1]:
# Filter Operations
# &, |, ==
# ~

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master('local[1]').appName('dataframe').getOrCreate()

In [4]:
data = spark.read.csv('forecasting_mini_course_sales/train.csv', header=True, inferSchema=True)
data.show(4)

+---+----------+---------+------------+--------------------+--------+
| id|      date|  country|       store|             product|num_sold|
+---+----------+---------+------------+--------------------+--------+
|  0|2017-01-01|Argentina|Kaggle Learn|Using LLMs to Imp...|      63|
|  1|2017-01-01|Argentina|Kaggle Learn|Using LLMs to Tra...|      66|
|  2|2017-01-01|Argentina|Kaggle Learn|Using LLMs to Win...|       9|
|  3|2017-01-01|Argentina|Kaggle Learn|Using LLMs to Win...|      59|
+---+----------+---------+------------+--------------------+--------+
only showing top 4 rows



### Filter Operations

In [6]:
### sales units less than 3
data.filter("num_sold<3").show(5)

+-----+----------+---------+------------+--------------------+--------+
|   id|      date|  country|       store|             product|num_sold|
+-----+----------+---------+------------+--------------------+--------+
|87377|2020-03-11|Argentina|Kaggle Learn|Using LLMs to Win...|       2|
|88952|2020-04-01|Argentina|Kaggle Learn|Using LLMs to Win...|       2|
|89027|2020-04-02|Argentina|Kaggle Learn|Using LLMs to Win...|       2|
|89102|2020-04-03|Argentina|Kaggle Learn|Using LLMs to Win...|       2|
|89327|2020-04-06|Argentina|Kaggle Learn|Using LLMs to Win...|       2|
+-----+----------+---------+------------+--------------------+--------+
only showing top 5 rows



In [8]:
### get country, store and product where sales units less than 5 
data.filter("num_sold<5").select(['country', 'store', 'product']).show(4)

+---------+------------+--------------------+
|  country|       store|             product|
+---------+------------+--------------------+
|Argentina|Kaggle Learn|Using LLMs to Win...|
|Argentina|Kaggle Learn|Using LLMs to Win...|
|Argentina|Kaggle Learn|Using LLMs to Win...|
|Argentina|Kaggle Learn|Using LLMs to Win...|
+---------+------------+--------------------+
only showing top 4 rows



In [17]:
### multiple filters
data.filter((data['country']=='Argentina') & (data['num_sold']>400)).show()

+-----+----------+---------+---------+--------------------+--------+
|   id|      date|  country|    store|             product|num_sold|
+-----+----------+---------+---------+--------------------+--------+
|27160|2017-12-29|Argentina|Kagglazon|Using LLMs to Imp...|     411|
+-----+----------+---------+---------+--------------------+--------+



In [22]:
### filters - inverse condition
data.filter(~(data['num_sold']<900)).show(5)

+---+----------+-------+---------+--------------------+--------+
| id|      date|country|    store|             product|num_sold|
+---+----------+-------+---------+--------------------+--------+
| 25|2017-01-01| Canada|Kagglazon|Using LLMs to Imp...|    1041|
| 26|2017-01-01| Canada|Kagglazon|Using LLMs to Tra...|    1101|
| 28|2017-01-01| Canada|Kagglazon|Using LLMs to Win...|    1062|
| 55|2017-01-01|  Japan|Kagglazon|Using LLMs to Imp...|     938|
| 58|2017-01-01|  Japan|Kagglazon|Using LLMs to Win...|     970|
+---+----------+-------+---------+--------------------+--------+
only showing top 5 rows



### Aggregation Functions

In [23]:
data.printSchema()

root
 |-- id: integer (nullable = true)
 |-- date: date (nullable = true)
 |-- country: string (nullable = true)
 |-- store: string (nullable = true)
 |-- product: string (nullable = true)
 |-- num_sold: integer (nullable = true)



In [31]:
## Country wise average sales across time
data.groupBy('country').avg('num_sold').show()

+---------+------------------+
|  country|     avg(num_sold)|
+---------+------------------+
|Argentina| 61.45469149324571|
|    Spain|159.94384811975172|
|  Estonia|130.60675428988682|
|   Canada| 256.0465133260314|
|    Japan| 219.5613727637824|
+---------+------------------+



In [32]:
### Group by store where max sales
data.groupBy('store').sum('num_sold').show()

+------------+-------------+
|       store|sum(num_sold)|
+------------+-------------+
|Kaggle Learn|      2755934|
|   Kagglazon|     15661335|
|Kaggle Store|      4251056|
+------------+-------------+



In [37]:
### count of product
data.groupBy('product').count().show()

+--------------------+-----+
|             product|count|
+--------------------+-----+
|Using LLMs to Imp...|27390|
|Using LLMs to Tra...|27390|
|Using LLMs to Win...|27390|
|Using LLMs to Wri...|27390|
|Using LLMs to Win...|27390|
+--------------------+-----+



In [40]:
### apply aggregation on specific columns
data.groupBy('country').agg({'date': 'min', 'num_sold': 'sum'}).show()

+---------+----------+-------------+
|  country| min(date)|sum(num_sold)|
+---------+----------+-------------+
|Argentina|2017-01-01|      1683244|
|    Spain|2017-01-01|      4380862|
|  Estonia|2017-01-01|      3577319|
|   Canada|2017-01-01|      7013114|
|    Japan|2017-01-01|      6013786|
+---------+----------+-------------+

